In [2]:
import pandas as pd
from datetime import datetime
import bcrypt

In [3]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [4]:
df = pd.read_csv('csv/users.csv', encoding='utf-8')

In [5]:
df['num_of_reviews'] = df[['Review1', 'Review2', 'Review3', 'Review4', 'Review5']].notnull().sum(axis=1)

In [6]:
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [7]:
df = df.drop(['Profile picture', 'Review1', 'Review2', 'Review3', 'Review4', 'Review5', 'Review6', 'Shutterstock URL', 'Photo ID'], axis = 1)

In [8]:
df['Username'] = df['Username'].str.replace('\n', '')

In [9]:
df.rename(columns = {'PUBLIC':'hasPublicProfile', 'Nationality' : 'nationality', \
                    'Age' : 'age', 'Sex' : 'sex', 'About Me' : 'bio', 'Name' : 'firstName', \
                    'Surname' : 'lastName', 'Username' : 'username'}, inplace = True)

In [10]:
columns = ['id', 'createdDate', 'updatedDate', 'isActive', 'email', 'hash', 'hasPaswordSaved', \
          'phone', 'statusId', 'defaultLocalityId', 'coverImageId', 'level', 'passwordResetHash', 'passwordResetValid', \
          'searchVector', 'searchConfig', 'defaultCurrencyId', 'imageId']

In [11]:
df = pd.concat([df, pd.DataFrame(columns = columns)])

In [12]:
def set_public_profile(value):
    if value == 'ANO':
        return True
    return False

In [13]:
def create_pass(value):
    temp = 'Trynego+' + str(value).lower()
    bytePwd = temp.encode('utf-8')
    mySalt = bcrypt.gensalt(10)
    hash = bcrypt.hashpw(bytePwd, mySalt)
    return hash.decode("utf-8")

In [14]:
def get_level_id(value):
    temp = int(value) * 2
    if temp < 15:
        return 1
    elif temp < 60:
        return 2
    elif temp < 250:
        return 3
    elif temp < 1000:
        return 4
    elif temp < 3500:
        return 5
    else:
        return 6

In [15]:
df['id'] = [i for i in range(1, df.shape[0]+1)]
df['order'] = ''
df['createdDate'] = dt_string
df['updatedDate'] = dt_string
df['hasPublicProfile'] = df['hasPublicProfile'].apply(set_public_profile)
df['age'] = df.age.astype(int)
df['sex'] = df.sex.str.lower()
df['statusId'] = 4
df['level'] = df['num_of_reviews'].apply(get_level_id)
df['defaultCurrencyId'] = 'USD'

In [16]:
df['hash'] = create_pass('Travelling')

In [17]:
df['isActive'] = True
df['email'] = df.firstName.str.lower() + '.' + df.lastName.str.lower() + '@trynego.com'

In [18]:
def set_image_id(value):
    try:
        return image[:][image['name'] == str(value)]['id'].values[0]
    except:
        return ''

In [19]:
df['imageId'] = df['Photo name'].apply(set_image_id)

In [20]:
df = df.drop(['Photo name'], axis = 1)

In [21]:
df = df.drop_duplicates(subset='username', keep="last")
df = df.drop_duplicates(subset='email', keep="last")

In [22]:
df.shape

(2447, 31)

In [23]:
df = df.drop(['ID'], axis=1)

In [24]:
df.to_csv('out/users.csv', index=False, encoding='utf-8')

In [25]:
df

,hasPublicProfile,nationality,age,sex,bio,firstName,lastName,username,8882,new_name,...,defaultLocalityId,coverImageId,level,passwordResetHash,passwordResetValid,searchVector,searchConfig,defaultCurrencyId,imageId,order
0,True,United Kingdom,26,male,"Hello there, this is Bruno, I'm a fullstack so...",Bruno,Turner,BrunoTurner,NaN,4330brunoturner,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
1,True,United Kingdom,32,male,"I'm Noel, a dedicaded sex and sexuality based ...",Noel,James,JamesN,NaN,9816jamesn,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
2,True,United Kingdom,30,male,"Hi, this is Abel, I'm an international investo...",Abel,Lee,Alee,NaN,3936alee,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,612,
3,True,United Kingdom,24,male,"In my childhood days, my love for observing th...",Seth,Scott,Scott007,NaN,0794scott007,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
4,True,United Kingdom,25,male,My collegues call me Chief Chef Aaron although...,Aaron,Hunter,Little.Hunter,NaN,0110littlehunter,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,True,United States,23,male,"Throw your dreams into space like a kite, and ...",Miguel,Mayo,Miguel.M,NaN,nanmiguelm,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
2444,True,United States,24,male,You have brains in your head. You have feet in...,Callen,Hopkins,Callen.Hacker,NaN,nancallenhacker,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
2445,True,United States,28,male,"For me, the voice of the sea speaks to the sou...",Harrison,Nunez,Famous.Harry,NaN,nanfamousharry,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,
2446,True,United Kingdom,28,male,I'm so beguiled with this: The world is mud-lu...,Quinn,Johnson,QuinnJohn,NaN,nanquinnjohn,...,NaN,NaN,1,NaN,NaN,NaN,NaN,USD,,


In [26]:
one = df.head(1)
one.to_csv('out/users_one.csv', index=False, encoding='utf-8')